# Smooth Cursor Curve

In [ ]:
from sympy import *

init_printing()

x = symbols('x')

## Simple Cubic Function

As an initial idea, we can use a simple cubic function to generate a smooth curve, instead of the linear function that is used by default.

In [ ]:
f = x ** 3

plot(f, (x, -1, 1))

## Fitting a Tangens Function

The cubic function is not a good fit for the cursor movement. Instead, we can use a tangens function to generate a smooth curve. For this we define some points that the curve should pass through and then fit a tangens function to these points.

In [ ]:
a, b = symbols("a b")

p1 = [1, 1]
p2 = [0.5, 0.3]

f_tan = a * tan(b * x)

f_tan1 = f_tan.subs(x, p1[0]) - p1[1]
f_tan2 = f_tan.subs(x, p2[0]) - p2[1]

sol = solve([f_tan1, f_tan2], [a, b])
display(sol)

for s in sol:
    f_tan = f_tan.subs(a, s[0]).subs(b, s[1])
    display(f_tan)
    plot(f_tan, (x, -1, 1))

## Polynomial Regression

Another approach is to use polynomial regression to fit a curve to the points. This is a more general approach than the tangens function, but it is also more complex. For this we use the `numpy` library.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define initial points and separate into px and py
pp = np.array([[0, 0], [0.25, 0.1], [0.5, 0.25], [0.75, 0.7], [1, 1]])
px, py = pp[:, 0], pp[:, 1]

# Add negative counterparts and sort
px = np.concatenate((px, -px[1:]))
py = np.concatenate((py, -py[1:]))
sort_idx = np.argsort(px)
px, py = px[sort_idx], py[sort_idx]

# Remove duplicates
_, unique_idx = np.unique(px, return_index=True)
px, py = px[unique_idx], py[unique_idx]

# run the regression
degree = 5
p = np.polyfit(px, py, degree)

# plot the results
xx = np.linspace(-1, 1, 100)
yy = np.polyval(p, xx)

plt.plot(xx, yy, label='polyfit(deg={})'.format(degree))
plt.plot(xx, xx, '--', label='y=x')
plt.plot(px, py, 'xr', label='points')

# Axis adjustments
plt.axis([-1, 1, -1, 1])
plt.legend(loc='best')
ax = plt.gca()
ax.set_aspect('equal', adjustable='box')

# Move axis lines to zero point
ax.spines['left'].set_position('zero')
ax.spines['bottom'].set_position('zero')
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')

## Sigmoid Function

### Plain Sigmoid
Observing a plain sigmoid function we can see, that over a range of -1 to 1 we have a slow increase at the beginning, a fast increase in the middle and a slow increase at the end. This is exactly what we want for the cursor movement. We can use the sigmoid function to generate a smooth curve.

In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

x = sp.Symbol('x')

FACTOR = 3

def sigmoid(x):
    return 1 / (1 + sp.exp(-x))

def plot_sigmoid(expr):
    # Convert the SymPy expression to a Python function
    f_sig = sp.lambdify(x, expr, 'numpy')
    
    xx = np.linspace(-1, 1, 100)
    yy = f_sig(xx)
    plt.plot(xx, yy, label=f'sigmoid')
    plt.plot(xx, xx, '--', label='y=x')
    plt.axis([-1, 1, -1, 1])
    plt.legend(loc='best')
    ax = plt.gca()
    ax.set_aspect('equal', adjustable='box')
    ax.spines['left'].set_position('zero')
    ax.spines['bottom'].set_position('zero')
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    
    display(expr)

plot_sigmoid(sigmoid(FACTOR*x))
plt.show()


### Adjusting the sigmoid function

We need our sigmoid function to only happen in the top right corner and then mirror it to the negative. For this we need to adjust the function. We can do this by adding a constant to the x value and then multiplying the result by a constant. This will shift the sigmoid function to the right and then scale it to the desired size.

In [ ]:
positive_sigmoid = sigmoid(FACTOR*(2*x-1))

plot_sigmoid(positive_sigmoid)

### Hitting the Edges

The sigmoid function will never reach 0 or 1, as it only converges towards these limits. Therefore, we need to adjust it to hit the edges.

In [ ]:
edge = (sigmoid(FACTOR*(2*x-1)) - sigmoid(-FACTOR))/(sigmoid(FACTOR) - sigmoid(-FACTOR))

plot_sigmoid(edge)

### Rotate into negative domain

We need to mirror the function into the negative domain, since we want the cursor to move in the negative direction as well. For this we need to cut the function in half and mirror it to the negative. We can use a 180 degree rotation transformation:

$$
(x, y) \rightarrow (-x, -y)
$$

In [ ]:
# rotate 180 degrees
rotated_edge = - edge.subs(x, -x)

plot_sigmoid(rotated_edge)

In [ ]:
# combine the two edges
combined_edge = sp.Piecewise((edge, x >= 0), (rotated_edge, x < 0))

plot_sigmoid(combined_edge)

### Now lets abstract this and try out multiple FACTORS

After we have found a good sigmoid function, we can abstract it and try out multiple factors. This will allow us to find the best factor for the cursor movement. Our sigmoid function will now look like this:

In [ ]:
def combined_edge_sigmoid(factor):
    positive_sigmoid = sigmoid(factor*(2*x-1))
    edge = (sigmoid(factor*(2*x-1)) - sigmoid(-factor))/(sigmoid(factor) - sigmoid(-factor))
    rotated_edge = - edge.subs(x, -x)
    combined_edge = sp.Piecewise((edge, x >= 0), (rotated_edge, x < 0))
    return combined_edge

factors = [1, 2, 3, 4, 5, 6]

# plot the results into a 3x3 grid
fig, axs = plt.subplots(2, 3, figsize=(15, 15))
for i, factor in enumerate(factors):
    plt.subplot(2, 3, i+1)
    plot_sigmoid(combined_edge_sigmoid(factor))
    plt.title(f'factor={factor}')

plt.show()